In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import math 
from sklearn.metrics import mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pmdarima

# Temperature Analysis on the DailyClimate Time Serie

In this notebook we analyze the 'meantemp' feature looking at the possible approaches to fit the series in order to make forecasting of the future values. 
After the preparation of the dataset, we discuss and evaluate 4 different approaches:
- SARIMAX
- auto_arima
- Linear Regression
- Random Forests


# *Preparation of the dataset*

Let's consider only the meantemp feature
In this part we adapt the dataset in order to make this feature analyzible as a time serie with models like ARIMA and SARIMAX

In [ ]:
#concatenation of the two datasets (train and test) into a single one
train_climate = pd.read_csv('/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv')
test_climate = pd.read_csv('/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTest.csv')
daily_climate = pd.concat([train_climate, test_climate], axis=0)
daily_climate.head()

In [ ]:
train_climate

In [ ]:
test_climate

In [ ]:
#check if date is already a DateTime object
daily_climate.info()

In [ ]:
daily_climate.describe()

In [ ]:
#convert date as a DateTime object
daily_climate['date']=pd.to_datetime(daily_climate['date'])

In [ ]:
#set date as the key of the dataset
daily_climate.set_index(daily_climate['date'], inplace=True)
daily_climate = daily_climate.drop('date', axis=1)
daily_climate.head()

In [ ]:
#focus only on the temperature
daily_climate = daily_climate.drop(['humidity', 'wind_speed', 'meanpressure'], axis=1)
daily_climate.head()

In [ ]:
#plot the train and test data we have preparated

plt.figure(figsize=(5,5))
sns.lineplot(daily_climate['meantemp'].iloc[:train_climate.shape[0]], color='blue', label='Train data')
sns.lineplot(daily_climate['meantemp'].iloc[-test_climate.shape[0]:], color='red', label='Test data')
selected_dates = daily_climate.index[::100]
plt.xticks(selected_dates, rotation=45)
plt.title('Temperature Serie')
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
#check if it is a time serie by looking at the autocorrelation
print(daily_climate['meantemp'].autocorr())

# First model : SARIMAX  
We use **SARIMAX** as a model to predict the time serie of the temperature.
As input we put the train data.
The order of the model chosen is (3,1,1) which means, respectively:
- order 3 of the autoregressive part (look a temperature of yesterday, two days ago and three days ago
- order 1 of the differences (compute first differences that is Today's Temp - Yesterday's Temp) 
- order 1 of the moving average part (model only yesterday's noise)


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

#daily_climate.index.freq = 'D'

temp_model = SARIMAX(daily_climate['meantemp'].iloc[:train_climate.shape[0]], order = (3,1,1))
temp_result = temp_model.fit()
temp_prediction = temp_result.get_prediction()
temp_mean_prediction = temp_prediction.predicted_mean
plt.plot(daily_climate['meantemp'].iloc[:train_climate.shape[0]],label='data')
temp_mean_prediction.plot(color='red', label='forecast')
plt.legend()
plt.show()


In [ ]:
forecast_steps = len(test_climate)  # Number of steps for the forecasting
forecast = temp_result.get_forecast(steps=forecast_steps)
mean_forecast = forecast.predicted_mean

plt.plot(daily_climate['meantemp'].iloc[-test_climate.shape[0]:],label='data')
mean_forecast.plot(color='red', label='forecast')
plt.legend()
plt.show()



This is not an impressive forecasting

In [ ]:
rmse = math.sqrt(mean_squared_error(daily_climate["meantemp"].iloc[-test_climate.shape[0]:], temp_result.forecast(steps=forecast_steps)))
print("The root mean squared error is %.3f"%rmse)

# Second model : auto_arima

An auto_arima model tries to learn itself what are the best parameters to apply at the ARIMA model and then performs the fitting using these parameters. 

In [ ]:
import pmdarima

arima_model = pmdarima.auto_arima(daily_climate['meantemp'].iloc[:train_climate.shape[0]], seasonal=True, m=12)
forecasts = arima_model.predict(forecast_steps)

plt.plot(daily_climate['meantemp'].iloc[-test_climate.shape[0]:],label='data')
plt.plot(forecasts, color='red', label='forecast')
plt.xticks(rotation=45)
plt.legend()
plt.show()

Also this approach is not satisfying

In [ ]:
rmse = math.sqrt(mean_squared_error(daily_climate["meantemp"].iloc[-test_climate.shape[0]:], forecasts))
print("The root mean squared error is %.3f"%rmse)

# *Switching to traditional models ...*

To deal with time series analysis using Linear Regression or Random Forests we must preparate the dataset first. This means to have on the level of the single data points the information needed.   In our case, we try to see what happens using only the information about Yesterday's temperature


In [ ]:
daily_climate_ = daily_climate.copy(deep=True)
daily_climate_['Yesterday']=daily_climate_['meantemp'].shift()
daily_climate_.dropna(inplace=True)
daily_climate_.head()

In [ ]:
y = daily_climate_['meantemp']
X = daily_climate_['Yesterday']

In [ ]:
X_train = X[: '2017-01-01']
X_test = X ['2017-01-01':]
y_train = y[:'2017-01-01']
y_test = y['2017-01-01':]

In [ ]:
plt.plot(y_train,label='Train')
plt.plot(y_test,label='Test')
plt.xticks(rotation=45)
plt.legend()
plt.show()

# Third model : Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train.values.reshape(-1,1),y_train)

yt = lr_model.predict(X_train.values.reshape(-1,1))
yp = lr_model.predict(X_test.values.reshape(-1,1))

In [ ]:
df_result_train = pd.DataFrame(y_train)
df_result_train['predicted'] = yt
df_result_test = pd.DataFrame(y_test)
df_result_test['predicted'] = yp

fig,axes = plt.subplots(1,2,figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(df_result_train['meantemp'],label='Data')
plt.plot(df_result_train['predicted'],label='Predicted')
plt.xticks(rotation=45)
plt.title("Forecasting on Training Set")
plt.legend();

plt.subplot(1,2,2)
plt.plot(df_result_test['meantemp'],label='Data')
plt.plot(df_result_test['predicted'],label='Predicted')
plt.xticks(rotation=45)
plt.title("Forecasting on Test Set : RMSE = %3.f"%math.sqrt(mean_squared_error(yp,y_test)))
plt.legend();

This prediction is the most satisfying so far

# Fourth model : Random Forest Regressor 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=500)
rf_model.fit(X_train.values.reshape(-1,1), y_train)

yt = rf_model.predict(X_train.values.reshape(-1,1))
yp = rf_model.predict(X_test.values.reshape(-1,1))

In [ ]:
df_result_train = pd.DataFrame(y_train)
df_result_train['predicted'] = yt
df_result_test = pd.DataFrame(y_test)
df_result_test['predicted'] = yp
#%%
fig,axes = plt.subplots(1,2,figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(df_result_train['meantemp'],label='Data')
plt.plot(df_result_train['predicted'],label='Predicted')
plt.xticks(rotation=45)
plt.title("Forecasting on Training Set")
plt.legend();

plt.subplot(1,2,2)
plt.plot(df_result_test['meantemp'],label='Data')
plt.plot(df_result_test['predicted'],label='Predicted')
plt.xticks(rotation=45)
plt.title("Forecasting on Test Set : RMSE = %.3f"%math.sqrt(mean_squared_error(yp,y_test)))
plt.legend();

This prediction is quite similar to the one obtained with Linear Regression, also notice the value of the rmse are very similar